# Transformer

## Introdução

### RNNs clássicas para NLP

- Palavras são codificadas em vetores
- Cada novo estado é baseado no estado anterior
- Decodificação começa no estado final do codificador
![Rnns_classica](images/rnn_classica.png)

### Mecanismo de atenção

- ultima camada do codificador sobrecarregada com todos os textos
- atenção maior para as camadas
- maiores pesos para o contexto estado anterior

![Mecanismo de atenção](images/attention.png)

## Arquitetura Transformer

Essa arquitetura tem como foco os mecanismos de atenção. Ao em vez de cada palavra receber a atenção, nos Transformers cada frase terá esse processo.

![arquitetura](images/transformer.png)

In [1]:
import pandas as pd
import numpy as np